# Free energy calculations of System 3

## Trial 1: 1D + 2D alchemical metadynamics (serial manner)

Trial 1 is a 200 ns simulation, whose first 100 ns is a 1D alchemical metadynamics followed by a 100 ns 2D alchemical metadynamics. Below is the content of the PLUMED input file. 

In [1]:
%cat trial_1/plumed.dat

center: CENTER ATOMS=1-144            # geometric center of the host molecule
water_group: GROUP ATOMS=207-6656:3   # oxygen atom of the water molecules
n: COORDINATION GROUPA=center GROUPB=water_group R_0=0.35
lambda: EXTRACV NAME=lambda

METAD ...
ARG=lambda 
SIGMA=0.01     
HEIGHT=12.394781044629076
PACE=10        
GRID_MIN=0    
GRID_MAX=39    
GRID_BIN=39
TEMP=298
BIASFACTOR=150
LABEL=metad_1   
UPDATE_UNTIL=100000
FILE=HILLS_1
... METAD

METAD ...
ARG=lambda,n
SIGMA=0.01,0.05    
HEIGHT=12.394781044629076
PACE=100
GRID_MIN=0,0    
GRID_MAX=39,20   
GRID_BIN=39,100
TEMP=298
BIASFACTOR=30
LABEL=metad_2
UPDATE_FROM=100000
FILE=HILLS_2
... METAD

UPPER_WALLS ...
 ARG=n
 AT=10.5
 KAPPA=200.0
 EXP=2
 EPS=1
 OFFSET=0
 LABEL=uwall
... UPPER_WALLS

LOWER_WALLS ...
 ARG=n
 AT=0.7
 KAPPA=200.0
 EXP=2
 EPS=1
 OFFSET=0
 LABEL=lwall
... LOWER_WALLS

PRINT STRIDE=10 ARG=* FILE=COLVAR



To calculate the free energy difference between the coupled and uncoupled states, I followed the following procedures:
- For the two `HILLS` files, I ran `sum_hills` and plotted the free energy difference as a function of time. Below are the commands I used for each file. `fes_*.dat` files were generated every 20 ps. (Left figure: 1D; right figure: 2D)
  - For `HILLS_1`: `plumed sum_hills --hills HILLS_1 --min 0 --max 39 --bin 39 --mintozero --stride 1000`
  - For `HILLS_2`: `plumed sum_hills --hills HILLS_2 --min 0,0 --max 39,20 --bin 39,100 --idw lambda --kt 2.4777090399459767 --mintozero --stride 100`
<img src=https://i.imgur.com/8bdPLY0.png width=800>
- Both figures look worrisome, especially the 2D part, but I continued the analysis with the following commands. (Both Python codes and `plumed_sum_bias_*.dat` are attached. The free energy method implemented in the Python code is the same as [Check.ipynb](https://github.com/wehs7661/lambda_MetaD_questions/blob/master/archived_questions/210314_questions/Method_1/Check.ipynb), which is also the method we reported in the paper. The only difference between the two codes is that `calculate_df_2D.py` additionally considered the wall biases when reweighting.) 
  - `python calculate_df_1D.py -d ./ -n 20 -hh HILLS_1 -t 0.5 -a 0.5 -i plumed_sum_bias_1.dat` (truncation: 0.5, average fraction: 0.5)
  - `python calculate_df_2D.py -d ./ -n 20 -hh HILLS_2 -t 0.4 -a 0.6 -i plumed_sum_bias_2.dat` (truncation: 0.4, average fraction: 0.6)

As a result, the free energy difference (between the coupled and uncoupled states) estimated by the two parts are 132.78 +/- 0.24 kT and 7.56 +/- 0.65 kT, respectively. As the 2D simulation deposited Gaussian biases upon the free energy surface that had been biased by the 1D simulation, the free energy difference estimated by the whole simulation should just be the sum of the two values, which is 140.34 kT. The inconsistency of this value compared to the reference (~148 kT) is certainly what we need to address.

## Trial 2: 1D + 2D alchemical metadynamics (concurrent manner)

In Trial 2, 1D and 2D alchemical metadynamics were performed simultaneously for 200 ns, using the following PLUMED input file.

In [2]:
%cat trial_2/plumed.dat

center: CENTER ATOMS=1-144            # geometric center of the host molecule
water_group: GROUP ATOMS=207-6656:3   # oxygen atom of the water molecules
n: COORDINATION GROUPA=center GROUPB=water_group R_0=0.35
lambda: EXTRACV NAME=lambda

METAD ...
ARG=lambda 
SIGMA=0.01     
HEIGHT=12.394781044629076
PACE=10        
GRID_MIN=0    
GRID_MAX=39    
GRID_BIN=39
TEMP=298
BIASFACTOR=150
LABEL=metad_1   
FILE=HILLS
... METAD

METAD ...
ARG=lambda,n
SIGMA=0.01,0.05    
HEIGHT=4.955418079891953
PACE=500
GRID_MIN=0,0    
GRID_MAX=39,20   
GRID_BIN=39,100
TEMP=298
BIASFACTOR=30
LABEL=metad_2
FILE=HILLS
... METAD

UPPER_WALLS ...
 ARG=n
 AT=10.5
 KAPPA=200.0
 EXP=2
 EPS=1
 OFFSET=0
 LABEL=uwall
... UPPER_WALLS

LOWER_WALLS ...
 ARG=n
 AT=0.7
 KAPPA=200.0
 EXP=2
 EPS=1
 OFFSET=0
 LABEL=lwall
... LOWER_WALLS

PRINT STRIDE=10 ARG=* FILE=COLVAR



In this case, I accidentally output the HILLS information of 1D and 2D simulations to the same HILLS file, but I've separated them as `HILLS_1` and `HILLS_2`. Below are the time series of the free energy difference obtained from `sum_hills`.

<img src=https://i.imgur.com/509S8QC.png width=1000>

In the figure above, the left and middle panels are the free energy difference obtained from `HILLS_1` and `HILLS_2`, respectively. The right panel is the sum of the two. Based on the right figure, I adopted a truncation fraction of 0.4 and an average fraction of 0.6. Then, I executed the following command: `python calculate_df.py -d ./ -n 20 -hh HILLS_1 HILLS_2 -t 0.4 -a 0.6 -i plumed_sum_bias.dat`. The code `calculate_df.py` is attached, whose difference from the codes in trial 1 is that it could take into 2 HILLS files and consider the sum of `metad_1.bias`, `metad_2.bias`, `uwall.bias`, and `lwall.bias` during reweighting. As a result, the estimated free energy difference was 134.06 +/- 0.40 kT, which is still far from the reference.

## Additional analysis of Trials 1 and 2

To better understand Trials 1 and 2, I plotted the time series of the number of water molecules and the Gaussian height as below. 

### Trial 1
As shown below, the Gaussian height of the 1D simulation (left) was stationary in the end, while the bias in the 2D part was still large in the end. When the configurational bias was introduced, the time scale of water motion was significantly shortened. 

<img src=https://i.imgur.com/NAqoTnC.png width=1000>

### Trial 2
As shown below, the Gaussian height was even larger than that in the 2D part of Trial 1, and the time scale of the water motion was not as short as the one in Trial 1. This is probably because I used a larger pace (500 steps) for Gaussian deposition in Trial 2.  
<img src=https://i.imgur.com/Yo166eE.png width=1000>

To improve the quality of free energy calculations, now we are planning another simulation, which will make the following changes compared to the first two trials:
- We will set the initial Gaussian height for both 1D and 2D biases as 5 kT.
- The pace of 2D Gaussian deposition will be as fast as 1D (pace=10 steps).
- We will use wider Gaussians (width=0.3) based on the rule of thumb that the reasonable Gaussian width should be approximately the same as the standard deviation of the CV in an unbiased simulation.

## Our questions
Regarding the data shown above, we have the following two questions:
- Do you have any other suggestions for changes to be made in trial 3 to improve the free energy calculation of System 3? For your information, the PLUMED input files and Python codes are provided in folders such as `trial_1` and `trial_2` and the `HILLS` files and `COLVAR` files can be obtained from [this link](https://drive.google.com/file/d/1Qkfc21eOsTW5H1kc7tmzUbmRR3uPjo9V/view?usp=sharing).
- We are also considering a distance restraint between the host and guest molecule using PLUMED. Ideally, we want to make the force constant of the harmonic restraint lambda-dependent such that it's gradually turned on while the van der Waals interaction is gradually decoupled (from states 22 to 40 in our case). The functional form of such a restraint could be as simple as $\frac{\lambda}{2}(d-d_0)^2$, where $\lambda$ is the coupling parameter of the van der Waals interaction that ranges from 0 to 1. However, in the current implementation of alchemical metadynamics, `lambda` in the PLUMED input file is the state index rather than coupling parameters. In our case, `lambda` corresponds to integers from 0 to 19 since we have 20 states. I'm wondering if it is possible to retrieve the corresponding coupling parameters from these state indices such that we could apply a $\lambda$-dependent restraint.